## Scenario 3: Multiple data scientists working on multiple ML models
MLflow setup:

- Tracking server: yes, remote server (EC2).
- Backend store: postgresql database.
- Artifacts store: s3 bucket.
- The experiments can be explored by accessing the remote server.

The exampe uses AWS to host a remote server. In order to run the example you'll need an AWS account. Follow the steps described in the file mlflow_on_aws.md to create a new AWS account and launch the tracking server.

In [ ]:
import mlflow
import os 

os.environ['AWS_PROFILE'] = ''
TRACKING_SERVER_URI = ''
mlflow.set_tracking_uri(f'http://{TRACKING_SERVER_URI}:5000')

In [ ]:
print(mlflow.get_tracking_uri())

In [ ]:
mlflow.search_experiments()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

In [ ]:
mlflow.search_experiments()

## Interacting with the model registry

In [ ]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_URI}:5000")

In [ ]:
TRACKING_SERVER_URI

In [ ]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id

In [ ]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

In [ ]:
client.search_registered_models()